In [ ]:
# OR-Tools, PyTorch 설치
!pip install torch
!pip install ortools

In [ ]:
# (Colab) 모델을 불러오기 위해 필요한 파일을 다운로드 받습니다.
!wget https://raw.githubusercontent.com/cm8908/ScienceTouch_JibumKim_TSP/main/Transformer-TSP50/my_model_packages.py

In [2]:
# 필요한 패키지들을 불러옵니다.
import torch  # 파이토치 
import matplotlib.pyplot as plt  # 시각화 패키지 (Matplotlib - pyplot)
from my_model_packages import TSP_net, compute_tour_length  # 모델과 경로 길이를 계산해주는 함수를 불러옵니다
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # GPU 사용이 가능한 경우 속도 향상을 위해 GPU를 사용합니다.

In [ ]:
N = 50  # 도시의 개수는 50개로 설정해주었습니다. 원하는 다른 숫자를 주어도 좋습니다.

# 50 개의 도시를 가진 랜덤 외판원 문제 생성
data = torch.rand(N, 2).to(device)

# 콘솔에 데이터와 각 데이터의 위치를 표시해볼까요?
print(data.cpu().numpy())
plt.scatter(data.cpu()[:,0], data.cpu()[:,1])

In [ ]:
# 딥러닝 모델인 Transformer 모델을 선언해줍니다. 아직 학습은 되지 않은 상태입니다.
model = TSP_net('linear', None, None, 2, 128, 512, 6, 2, 8, 1000).to(device)

# 경로 탐색을 시작합니다. (Transformer + Greedy 알고리즘)
tour, _, _, _ = model(data[None], 1, greedy=True, beamsearch=False)

print('=<탐색 결과 (경로)>=')
for i in range(tour.size(1)-1):
    print(f'{tour[0][i].item()} -> ', end='')
    if (i+1) % 10 == 0:
        print()
print(f'{tour[0][i+1].item()}')
print()

print('=<경로 길이>=')
tour_length = compute_tour_length(data[None], tour)
print(tour_length.item())

In [ ]:
# 경로 그려보기
plt.scatter(data.cpu()[:,0], data.cpu()[:,1])
sorted_data = data.cpu()[tour[0].cpu()]
plt.plot(sorted_data[:,0], sorted_data[:,1], color='black')
plt.show()

In [6]:
# 이상한 경로가 나타난 것처럼 보이는 이유는, 아직 모델이 학습되지 않은 상태입니다.
# 여기서 모델을 학습시키면 시간이 매우 오래 걸리기 때문에, 이전에 미리 학습을 시켜둔 모델을 불러와서 사용하겠습니다.
# 미리 학습된 Transformer 모델 체크포인트 파일 읽어오기
checkpoint = torch.load('checkpoint/transformer_tsp50_demo.pt', map_location=torch.device('cuda'))
model.load_state_dict(
    checkpoint
)
model = model.to(device)

In [ ]:
# 경로 탐색을 시작합니다! (Transformer + Greedy 알고리즘)
tour, _, _, _ = model(data[None], 1, greedy=True, beamsearch=False)

print('=<탐색 결과 (경로)>=')
for i in range(tour.size(1)-1):
    print(f'{tour[0][i].item()} -> ', end='')
    if (i+1) % 10 == 0:
        print()
print(f'{tour[0][i+1].item()}')
print()

print('=<경로 길이>=')
tour_length = compute_tour_length(data[None], tour)
print(tour_length.item())

In [ ]:
# 경로 그려보기
plt.scatter(data.cpu()[:,0], data.cpu()[:,1])
sorted_data = data.cpu()[tour[0].cpu()]
plt.plot(sorted_data[:,0], sorted_data[:,1], color='black')
plt.plot((sorted_data[-1,0], sorted_data[0,0]), (sorted_data[-1,1], sorted_data[0,1]), color='black')
plt.show()

In [ ]:
# 이전에 봤던 OR-Tools와도 결과를 비교해보겠습니다.
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
ortools_data = (data * 100).long()

# 데이터를 생성하는 함수
def create_data_model():
    data = {}
    data["distance_matrix"] = torch.cdist(ortools_data.float(), ortools_data.float()).long().cpu().numpy()
    data["num_vehicles"] = 1
    data["depot"] = 0
    return data

# 데이터를 생성하고 경로 탐색 매니저와 모델을 선언해줍니다.
data_model = create_data_model()
manager = pywrapcp.RoutingIndexManager(
    len(data_model["distance_matrix"]), data_model["num_vehicles"], data_model["depot"]
)
routing = pywrapcp.RoutingModel(manager)

# 거리를 반환하는 함수
def distance_callback(from_index, to_index):
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data_model["distance_matrix"][from_node][to_node]

# 경로 탐색 모델에 거리 함수를 등록해줍니다.
transit_callback_index = routing.RegisterTransitCallback(distance_callback)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# 이전에 봤던 것처럼 필요한 초기값들을 설정해주는 코드입니다.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
)

# 탐색된 경로를 콘솔에 표시해주는 함수
def print_solution(manager, routing, solution):
    index = routing.Start(0)
    plan_output = "OR-Tools가 찾은 경로:\n"
    route_distance = 0
    tour = [0]
    while not routing.IsEnd(index):
        plan_output += f" {manager.IndexToNode(index)} ->"
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        tour.append(index)
    plan_output += f" {manager.IndexToNode(index)}\n"
    # print(plan_output)
    del tour[-1]
    return torch.LongTensor(tour).to(device)[None]

# OR-Tools를 활용해 경로를 탐색하고 출력해봅시다.
# Transformer와 OR-Tools 중 어느 것이 더 좋은 경로를 찾을까요? 경로 길이를 비교해봅시다.
solution = routing.SolveWithParameters(search_parameters)
if solution:
    tour = print_solution(manager, routing, solution)

print('=<탐색 결과 (경로)>=')
for i in range(tour.size(1)-1):
    print(f'{tour[0][i].item()} -> ', end='')
    if (i+1) % 10 == 0:
        print()
print(f'{tour[0][i+1].item()}')
print()

print('=<경로 길이>=')
tour_length = compute_tour_length(data[None], tour)
print(tour_length.item())

# 경로 그려보기
plt.scatter(data.cpu()[:,0], data.cpu()[:,1])
sorted_data = data.cpu()[tour[0]]
plt.plot(sorted_data[:,0], sorted_data[:,1], color='black')
plt.plot((sorted_data[-1,0], sorted_data[0,0]), (sorted_data[-1,1], sorted_data[0,1]), color='black')
plt.show()